In [30]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px

In [31]:
def plot_windrose(df):
    bins = np.linspace(0,24,13)
    labels = [0,2,4,6,8,10,12,14,16,18,20,22]

    df['WS100BIN'] = pd.cut(df['WS100'], bins=bins, labels = labels)

    df_grouped = df.groupby(['WD100CARD','WS100BIN']).count().reset_index()
    
    wd_card=["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]

    wd_zeros = np.zeros(len(wd_card))
    df_all_wd_card = pd.DataFrame([wd_card, wd_zeros, wd_zeros])
    df_all_wd_card = df_all_wd_card.T
    df_all_wd_card.columns = ['WD100CARD', 'WS100BIN', 'FREQUENCY']
    
    data_wind = df_grouped[['WD100CARD', 'WS100BIN', 'ZONEID']]
    data_wind.columns = df_all_wd_card.columns
    #print(data_wind.head(50))

    datax = pd.concat([data_wind, df_all_wd_card], axis = 0)
    print(datax.head())
    datax = datax.sort_values(by='WD100CARD', key=card_sorter)
    ws_ls = np.linspace(0,25,26)
    wind_all_speeds = pd.DataFrame([['N']*len(ws_ls), ws_ls, np.zeros(len(ws_ls))]).T
    wind_all_speeds.columns = ['WD100CARD', 'WS100BIN', 'FREQUENCY']
    wind_all_speeds
    datax = pd.concat([wind_all_speeds, df_all_wd_card, data_wind], axis = 0)

    fig = px.bar_polar(datax, 
                   r="FREQUENCY", 
                   theta="WD100CARD",
                   color="WS100BIN", 
                   #template="plotly_black",
                   color_discrete_sequence= px.colors.sequential.Plasma_r)                      
    #fig.show()
    return fig


In [26]:
def card_sorter(column):
    """Sort function"""
    wd_card=["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]
    correspondence = {card: order for order, card in enumerate(wd_card)}
    return column.map(correspondence)

In [32]:
### TEST
DATA_PATH = '../data/GEFCom2014Data/Wind/raw_data_incl_features.csv'
data = pd.read_csv(DATA_PATH)
plot_windrose(data)

  WD100CARD WS100BIN FREQUENCY
0         E        0       416
1         E        2      1514
2         E        4      2301
3         E        6      2294
4         E        8       660
